# Groupby-based method
This isn't working since groupby will always iterate in forward order, regardless of how its sliced.

In [ ]:
track = analysis_track.sel(time=slice('2023-08-30 12:18:00', '2023-08-30 12:23:00')).resample(time='50ms').interpolate()
cpa = uwacan.positional.closest_point(m_147_deploy, track)
reference = m_147_deploy

window_min_angle = 15
window_min_duration = None
window_min_length = None

angles = [-45, -30, -15, 0, 15, 30, 45]
angles = [0]

angles = np.sort(angles)
track = track.assign(angle=uwacan.positional.angle_between(reference, cpa, track))
if track.angle[0] > track.angle[-1]:
    # We want the angles to be negative before cpa and positive after
    track['angle'] *= -1

angles = xr.DataArray(angles, coords={'segment': angles})
center_indices = abs(angles - track.angle).argmin('time')
window_centers = track.isel(time=center_indices)#.assign_coords(track_index=center_indices)

# display(window_centers)

if angles.size > 1:
    actual_first_angle = track.angle.sel(time=window_centers.isel(segment=0).time)
    if abs(actual_first_angle - angles.isel(segment=0)) > abs(actual_first_angle - angles.isel(segment=1)):
        raise ValueError(f'Could not find window centered at {angles.isel(segment=0)}⁰, found at most {actual_first_angle}⁰.')
    actual_last_angle = track.angle.sel(time=window_centers.isel(segment=-1).time)
    if abs(actual_last_angle - angles.isel(segment=-1)) > abs(actual_first_angle - angles.isel(segment=-2)):
        raise ValueError(f'Could not find window centered at {angles.isel(segment=-1)}⁰, found at most {actual_last_angle}⁰.')

segments = []
for angle, segment_center in window_centers.groupby('segment'):
    print(f'{angle} at {segment_center.time.values}', flush=True)
    start_time = segment_center.time.values
    if window_min_angle:
        print(f'Finding angle from {start_time}', flush=True)
        for start_time, position in track.sel(time=slice(start_time, None, -1)).groupby('time'):
            angle_delta = uwacan.positional.angle_between(reference, segment_center, position)
            print(f'{angle_delta.values}')
            if abs(angle_delta) >= window_min_angle / 2:
                break
        else:
            raise ValueError(f'Start of window at {angle}⁰ not found in track. Not sufficiently high angles from window center.')
    if window_min_duration:
        print(f'Finding duration from {start_time}', flush=True)
        for start_time, position in track.sel(time=slice(start_time, None, -1)).groupby('time'):
            time_delta = (segment_center.time - position) / np.timedelta64(1, 's')
            if abs(time_delta) >= window_min_duration / 2:
                break
        else:
            raise ValueError(f'Start of window at {angle}⁰ not found in track. Not sufficient time from window center.')
    if window_min_length:
        print(f'Finding distance from {start_time}', flush=True)
        for start_time, position in track.sel(time=slice(start_time, None, -1)).groupby('time'):
            dist_delta = uwacan.positional.distance_between(segment_center, position)
            if abs(dist_delta) >= window_min_length / 2:
                break
        else:
            raise ValueError(f'Start of window at {angle}⁰ not found in track. Not sufficient distance from window center.')


    stop_time = segment_center.time.values
    if window_min_angle:
        print(f'Finding angle from {stop_time}', flush=True)
        for stop_time, position in track.sel(time=slice(stop_time, None)).groupby('time'):
            angle_delta = uwacan.positional.angle_between(reference, segment_center, position)
            print(f'{angle_delta.values}')
            if abs(angle_delta) >= window_min_angle / 2:
                break
        else:
            raise ValueError(f'End of window at {angle}⁰ not found in track. Not sufficiently high angles from window center.')
    if window_min_duration:
        print(f'Finding duration from {stop_time}', flush=True)
        for stop_time, position in track.sel(time=slice(stop_time, None)).groupby('time'):
            time_delta = (segment_center.time - position) / np.timedelta64(1, 's')
            if abs(time_delta) >= window_min_duration / 2:
                break
        else:
            raise ValueError(f'End of window at {angle}⁰ not found in track. Not sufficient time from window center.')
    if window_min_length:
        print(f'Finding distance from {stop_time}', flush=True)
        for stop_time, position in track.sel(time=slice(stop_time, None)).groupby('time'):
            dist_delta = uwacan.positional.distance_between(segment_center, position)
            if abs(dist_delta) >= window_min_length / 2:
                break
        else:
            raise ValueError(f'End of window at {angle}⁰ not found in track. Not sufficient distance from window center.')
    print(f'extracting from {start_time} to {stop_time}')
    segment_start, segment_stop = track.sel(time=start_time), track.sel(time=stop_time)
    segments.append(
        xr.concat([segment_start, segment_center, segment_stop], dim='time')
        .assign_coords(edge=('time', ['start', 'center', 'stop']))
        .swap_dims(time='edge')
        .assign(
            length=uwacan.positional.distance_between(segment_start, segment_stop),
            angle_span=segment_stop.angle - segment_start.angle,
            duration=(segment_stop.time - segment_start.time) / np.timedelta64(1, 's'),
        )
        .reset_coords('time')
    )

    print('proceeding', flush=True)

segments = xr.concat(segments, dim='segment')
display(segments)


# Loop and index version
This is slow since we have to index to the dataset all the time

In [ ]:
reference = m_147_deploy

window_min_angle = 15
# window_min_duration = 6
window_min_length = None

angles = [-45, -30, -15, 0, 15, 30, 45]

angles = np.sort(angles)
track = track.assign(angle=uwacan.positional.angle_between(reference, cpa, track))
if track.angle[0] > track.angle[-1]:
    # We want the angles to be negative before cpa and positive after
    track['angle'] *= -1

angles = xr.DataArray(angles, coords={'segment': angles})
center_indices = abs(angles - track.angle).argmin('time')
window_centers = track.isel(time=center_indices)#.assign_coords(track_index=center_indices)

# display(window_centers)

if angles.size > 1:
    actual_first_angle = track.angle.sel(time=window_centers.isel(segment=0).time)
    if abs(actual_first_angle - angles.isel(segment=0)) > abs(actual_first_angle - angles.isel(segment=1)):
        raise ValueError(f'Could not find window centered at {angles.isel(segment=0)}⁰, found at most {actual_first_angle}⁰.')
    actual_last_angle = track.angle.sel(time=window_centers.isel(segment=-1).time)
    if abs(actual_last_angle - angles.isel(segment=-1)) > abs(actual_first_angle - angles.isel(segment=-2)):
        raise ValueError(f'Could not find window centered at {angles.isel(segment=-1)}⁰, found at most {actual_last_angle}⁰.')

segments = []
for angle, segment_center in window_centers.groupby('segment'):
    print(angle)
    # Finding the start of the window
    center_idx = int(np.abs(track.time - segment_center.time).argmin())
    start_idx = center_idx
    if window_min_angle:
        while abs(segment_center.angle - track.isel(time=start_idx).angle) < window_min_angle / 2:
            start_idx -= 1
            if start_idx < 0:
                raise ValueError(f'Start of window at {angle}⁰ not found in track. Not sufficiently high angles from window center.')
    if window_min_length:
        while uwacan.positional.distance_between(segment_center, track.isel(time=start_idx)) < window_min_length / 2:
            start_idx -= 1
            if start_idx < 0:
                raise ValueError(f'Start of window at {angle}⁰ not found in track. Not sufficient distance from window center.')
    if window_min_duration:
        while abs(segment_center.time - track.time.isel(time=start_idx)) / np.timedelta64(1, 's') < window_min_duration / 2:
            start_idx -= 1
            if start_idx < 0:
                raise ValueError(f'Start of window at {angle}⁰ not found in track. Not sufficient time from window center.')
    # Finding the end of the window
    stop_idx = center_idx
    if window_min_angle:
        while abs(uwacan.positional.angle_between(reference, segment_center, track.isel(time=stop_idx))) < window_min_angle / 2:
            stop_idx += 1
            if stop_idx == track.sizes['time']:
                raise ValueError(f'End of window at {angle}⁰ not found in track. Not sufficiently high angles from window center.')
    if window_min_length:
        while uwacan.positional.distance_between(segment_center, track.isel(time=stop_idx)) < window_min_length / 2:
            stop_idx += 1
            if stop_idx == track.sizes['time']:
                raise ValueError(f'End of window at {angle}⁰ not found in track. Not sufficient distance from window center.')
    if window_min_duration:
        while abs(segment_center.time - track.time.isel(time=stop_idx)) / np.timedelta64(1, 's') < window_min_duration / 2:
            stop_idx += 1
            if stop_idx == track.sizes['time']:
                raise ValueError(f'End of window at {angle}⁰ not found in track. Not sufficient time from window center.')

    # Creating the window and saving some attributes
    if start_idx == stop_idx:
        segment = segment_center.assign(length=0, angle_span=0, duration=0)
    else:
        segment_start, segment_stop = track.isel(time=start_idx), track.isel(time=stop_idx)
        segment = (
            xr.concat([segment_start, segment_center, segment_stop], dim='time')
            .assign_coords(edge=('time', ['start', 'center', 'stop']))
            .swap_dims(time='edge')
            # .reset_coords('time')
            .assign(
                length=uwacan.positional.distance_between(segment_start, segment_stop),
                angle_span=segment_stop.angle - segment_start.angle,
                duration=(segment_stop.time - segment_start.time) / np.timedelta64(1, 's'),
            )
        )
    segments.append(segment.reset_coords('time'))
# segments[0]
segments = xr.concat(segments, dim='segment')
display(segments)

# Broadcasting solution
It's also possible to construct a method which calculates all the criteria for all segments at once, then does some fancy logic combination of condition bools to get masks for the regions you want as a (segment, time) dataarray.
From this array you should then find the time/index for the correct edges for each segment.